In [1]:
import nn_utils
import builders
import importlib

from ray import tune
import optuna
from ray.tune.suggest.optuna import OptunaSearch
import torch

from ray.tune.schedulers import AsyncHyperBandScheduler

import inspect
import argparse
import skorch
import os

from torch.utils import tensorboard
from sklearn import metrics

In [2]:
DATASETS = ["adult", "jasmine", "ldpa", "helena"]
AGGREGATORS = ["cls", "concatenate", "rnn"]
BATCH_SIZE = 128
SEED = 11

In [3]:
results = {}

In [4]:
for dataset_ in DATASETS:
    for aggregator_str_ in AGGREGATORS:
        
        dataset = dataset_
        aggregator_str = aggregator_str_
       
       
        print(f"Using -- Dataset:{dataset} Aggregator:{aggregator_str}")

        #####################################################
        # Configuration
        #####################################################

        MODULE = f"{dataset}.{aggregator_str}.config"
        CHECKPOINT_DIR = f"./{dataset}/{aggregator_str}/checkpoint"
        SEED = 11
        N_SAMPLES = 30

        BATCH_SIZE = 128
        MAX_EPOCHS = 1000
        EARLY_STOPPING = 30
        MAX_CHECKPOINTS = 10
        multiclass = False

        #####################################################
        # Util functions
        #####################################################

        def get_class_from_type(module, class_type):
            for attr in dir(module):
                clazz = getattr(module, attr)
                if callable(clazz) and inspect.isclass(clazz) and issubclass(clazz, class_type) and not str(clazz)==str(class_type):
                    return clazz

            return None

        def get_params_startswith(params, prefix):
            keys = [k for k in params.keys() if k.startswith(prefix)]
            extracted = {}

            for k in keys:
                extracted[k.replace(prefix, "")] = params.pop(k)

            return extracted


        def trainable(config, checkpoint_dir=CHECKPOINT_DIR):

            embedding_size = config.pop("embedding_size")

            encoders_params = get_params_startswith(config, "encoders__")
            aggregator_params = get_params_startswith(config, "aggregator__")
            preprocessor_params = get_params_startswith(config, "preprocessor__")

            model_params = {
                **config,
                "encoders": transformer_config.get_encoders(embedding_size, **encoders_params),
                "aggregator": transformer_config.get_aggregator(embedding_size, **aggregator_params),
                "preprocessor": transformer_config.get_preprocessor(**preprocessor_params),
                "optimizer": torch.optim.SGD,
                "criterion": criterion,
                "device": "cuda" if torch.cuda.is_available() else "cpu",
                "batch_size": BATCH_SIZE,
                "max_epochs": MAX_EPOCHS,
                "n_output": n_labels, # The number of output neurons
                "need_weights": False,
                "verbose": 1

            }

            if not os.path.exists(os.path.join(CHECKPOINT_DIR, "best_model/.fitted")):
                print("Not fitted before! I'm not going to do anything")
                return


            checkpoint = skorch.callbacks.Checkpoint(monitor="valid_loss_best", dirname=os.path.join(CHECKPOINT_DIR, "best_model"))

            model = nn_utils.build_transformer_model(
                        train_indices,
                        val_indices, 
                        [],
                        **model_params
                        )
            model.load_params(checkpoint=checkpoint)
            return model
        

            

        #####################################################
        # Dataset and components
        #####################################################

        module = importlib.import_module(MODULE)

        dataset = get_class_from_type(module, builders.DatasetConfig)
        if dataset is not None:
            dataset = dataset()
        else:
            raise ValueError("Dataset configuration not found")

        transformer_config = get_class_from_type(module, builders.TransformerConfig)
        if transformer_config is not None:
            transformer_config = transformer_config()
        else:
            raise ValueError("Transformer configuration not found")

        search_space_config = get_class_from_type(module, builders.SearchSpaceConfig)
        if search_space_config is not None:
            search_space_config = search_space_config()
        else:
            raise ValueError("Search space configuration not found")

        #####################################################
        # Configure dataset
        #####################################################

        if not dataset.exists():
            dataset.download()

        dataset.load(seed=SEED)

        preprocessor = nn_utils.get_default_preprocessing_pipeline(
                                dataset.get_categorical_columns(),
                                dataset.get_numerical_columns()
                            )

        #####################################################
        # Data preparation
        #####################################################

        train_features, train_labels = dataset.get_train_data()
        val_features, val_labels = dataset.get_val_data()
        test_features, test_labels = dataset.get_test_data()

        preprocessor = preprocessor.fit(train_features, train_labels)

        train_features = preprocessor.transform(train_features)
        val_features = preprocessor.transform(val_features)
        test_features = preprocessor.transform(test_features)

        all_features, all_labels, indices = nn_utils.join_data([train_features, val_features], [train_labels, val_labels])
        train_indices, val_indices = indices[0], indices[1]

        if dataset.get_n_labels() <= 2:
            n_labels = 1
            criterion = torch.nn.BCEWithLogitsLoss
        else:
            n_labels = dataset.get_n_labels()
            multiclass = True
            criterion = torch.nn.CrossEntropyLoss

        #####################################################
        # Hyperparameter search
        #####################################################
        try:
            analysis = tune.run(
                trainable,
                resume="AUTO",
                local_dir=CHECKPOINT_DIR, 
                name="param_search"    
            )

            best_config = analysis.get_best_config(metric="valid_loss", mode="min")
            del analysis
            print("Best config: ", best_config)

            model = trainable(best_config)
            y_pred = model.predict(test_features)

            if dataset_ not in results:
                results[dataset_] = {}

            if aggregator_str_ not in results[dataset_]:
                results[dataset_][aggregator_str_] = {}

            results[dataset_][aggregator_str_]["balanced_accuracy"] = metrics.balanced_accuracy_score(test_labels, y_pred)
            results[dataset_][aggregator_str_]["roc_auc"] = metrics.roc_auc_score(test_labels, y_pred)
            print(metrics.balanced_accuracy_score(test_labels, y_pred))
        except Exception as e:
            pass
        
        


Using -- Dataset:adult Aggregator:cls
Target mapping: {'<=50K': 0, '>50K': 1}
Numerical columns: ['fnlwgt', 'education-num']
Categorical columns: ['age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capitalgain', 'capitalloss', 'hoursperweek', 'native-country']
Columns: ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capitalgain', 'capitalloss', 'hoursperweek', 'native-country', 'class']


2022-03-17 00:19:32,919	INFO trial_runner.py:488 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2022-03-17 00:19:32,924	WARNING trial_runner.py:601 -- Attempting to resume experiment from /home/uriel/Documentos/CIC/DCC/Papers/Tabular Transformer/transformer-tabular-data/adult/cls/checkpoint/param_search. This will ignore any new changes to the specification.
2022-03-17 00:19:33,075	INFO tune.py:551 -- TrialRunner resumed, ignoring new add_experiment but updating trial resources.
2022-03-17 00:19:33,079	WARNING tune.py:580 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,dropout,embedding_size,n_head,n_hid,n_layers,optimizer__lr,iter,total time (s),train_loss,valid_loss,balanced_accuracy
trainable_727f4248,TERMINATED,192.168.1.72:2942,0.0699997,128,2,256,3,0.00909573,15,80.1114,0.333458,0.40196,0.816635
trainable_00535cf0,TERMINATED,192.168.1.72:2469,0.144154,256,2,32,3,0.008829,15,82.4752,0.332235,0.384565,0.819564
trainable_a0f55f9c,TERMINATED,192.168.1.72:2363,0.13851,1024,4,32,2,0.0129706,60,617.249,0.322936,0.360713,0.816545
trainable_c038bce2,TERMINATED,192.168.1.72:1519,0.000403666,1024,8,64,2,0.0119609,15,156.124,0.328601,0.450239,0.82022
trainable_85d079fa,TERMINATED,192.168.1.72:1351,0.00726525,1024,8,64,1,0.0322606,60,372.776,0.317215,0.343748,0.814118
trainable_25d5a1ec,TERMINATED,192.168.1.72:1084,0.0319958,1024,8,64,1,0.0395129,15,93.8465,0.391116,0.385768,0.779754
trainable_b3acad78,TERMINATED,192.168.1.72:481,0.048099,1024,8,64,2,0.0944024,15,157.05,0.324286,0.397083,0.813924
trainable_8a8a0242,TERMINATED,192.168.1.72:353,0.0503045,1024,8,64,2,0.065878,60,617.106,0.315559,0.347267,0.813576
trainable_616a1c6c,TERMINATED,192.168.1.72:32751,0.0504516,64,8,64,2,0.00253157,15,65.7435,0.408894,0.407552,0.754284
trainable_adfcbd1a,TERMINATED,192.168.1.72:32549,0.242383,64,8,512,2,0.00150805,15,65.047,0.532288,0.527636,0.5


2022-03-17 00:19:33,282	INFO tune.py:636 -- Total run time: 0.42 seconds (0.00 seconds for the tuning loop).


Best config:  {'n_layers': 2, 'optimizer__lr': 0.0643737674779829, 'n_head': 8, 'n_hid': 64, 'dropout': 0.00954326681844909, 'embedding_size': 1024}
0.7877334649080581
Using -- Dataset:adult Aggregator:concatenate
Target mapping: {'<=50K': 0, '>50K': 1}
Numerical columns: ['fnlwgt', 'education-num']
Categorical columns: ['age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capitalgain', 'capitalloss', 'hoursperweek', 'native-country']
Columns: ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capitalgain', 'capitalloss', 'hoursperweek', 'native-country', 'class']


...

2022-03-17 00:19:49,151	INFO trial_runner.py:488 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2022-03-17 00:19:49,156	WARNING trial_runner.py:601 -- Attempting to resume experiment from /home/uriel/Documentos/CIC/DCC/Papers/Tabular Transformer/transformer-tabular-data/helena/cls/checkpoint/param_search. This will ignore any new changes to the specification.
2022-03-17 00:19:49,170	ERROR trial_runner.py:359 -- [Errno 13] Permission denied: '/app'
2022-03-17 00:19:49,171	ERROR trial_runner.py:360 -- Runner restore failed.
Traceback (most recent call last):
  File "/home/uriel/Miniconda3/envs/DCC-attn/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 355, in __init__
    self.resume(run_errored_only=errored_only)
  File "/home/uriel/Miniconda3/envs/DCC-attn/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 611, in resume
    trials = load_trials_from_experiment_checkpoint(runner_state)
  File "/home/uriel/Minicon

Using -- Dataset:helena Aggregator:concatenate
Target mapping: {41: 0, 92: 1, 24: 2, 29: 3, 91: 4, 79: 5, 87: 6, 96: 7, 77: 8, 55: 9, 62: 10, 78: 11, 70: 12, 23: 13, 46: 14, 17: 15, 48: 16, 40: 17, 26: 18, 63: 19, 57: 20, 38: 21, 88: 22, 97: 23, 6: 24, 39: 25, 73: 26, 7: 27, 69: 28, 67: 29, 34: 30, 58: 31, 30: 32, 52: 33, 35: 34, 9: 35, 19: 36, 27: 37, 22: 38, 65: 39, 59: 40, 85: 41, 4: 42, 32: 43, 84: 44, 98: 45, 64: 46, 14: 47, 28: 48, 66: 49, 21: 50, 72: 51, 37: 52, 13: 53, 5: 54, 49: 55, 15: 56, 8: 57, 81: 58, 60: 59, 95: 60, 76: 61, 56: 62, 45: 63, 20: 64, 99: 65, 89: 66, 71: 67, 16: 68, 61: 69, 74: 70, 82: 71, 51: 72, 94: 73, 3: 74, 0: 75, 12: 76, 36: 77, 11: 78, 25: 79, 90: 80, 80: 81, 44: 82, 47: 83, 86: 84, 50: 85, 10: 86, 33: 87, 53: 88, 18: 89, 43: 90, 1: 91, 31: 92, 2: 93, 54: 94, 93: 95, 68: 96, 83: 97, 42: 98, 75: 99}
Numerical columns: ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21'

2022-03-17 00:19:49,568	INFO trial_runner.py:488 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2022-03-17 00:19:49,573	WARNING trial_runner.py:601 -- Attempting to resume experiment from /home/uriel/Documentos/CIC/DCC/Papers/Tabular Transformer/transformer-tabular-data/helena/concatenate/checkpoint/param_search. This will ignore any new changes to the specification.
2022-03-17 00:19:49,589	ERROR trial_runner.py:359 -- [Errno 13] Permission denied: '/app'
2022-03-17 00:19:49,589	ERROR trial_runner.py:360 -- Runner restore failed.
Traceback (most recent call last):
  File "/home/uriel/Miniconda3/envs/DCC-attn/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 355, in __init__
    self.resume(run_errored_only=errored_only)
  File "/home/uriel/Miniconda3/envs/DCC-attn/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 611, in resume
    trials = load_trials_from_experiment_checkpoint(runner_state)
  File "/home/uriel

Using -- Dataset:helena Aggregator:rnn
Target mapping: {41: 0, 92: 1, 24: 2, 29: 3, 91: 4, 79: 5, 87: 6, 96: 7, 77: 8, 55: 9, 62: 10, 78: 11, 70: 12, 23: 13, 46: 14, 17: 15, 48: 16, 40: 17, 26: 18, 63: 19, 57: 20, 38: 21, 88: 22, 97: 23, 6: 24, 39: 25, 73: 26, 7: 27, 69: 28, 67: 29, 34: 30, 58: 31, 30: 32, 52: 33, 35: 34, 9: 35, 19: 36, 27: 37, 22: 38, 65: 39, 59: 40, 85: 41, 4: 42, 32: 43, 84: 44, 98: 45, 64: 46, 14: 47, 28: 48, 66: 49, 21: 50, 72: 51, 37: 52, 13: 53, 5: 54, 49: 55, 15: 56, 8: 57, 81: 58, 60: 59, 95: 60, 76: 61, 56: 62, 45: 63, 20: 64, 99: 65, 89: 66, 71: 67, 16: 68, 61: 69, 74: 70, 82: 71, 51: 72, 94: 73, 3: 74, 0: 75, 12: 76, 36: 77, 11: 78, 25: 79, 90: 80, 80: 81, 44: 82, 47: 83, 86: 84, 50: 85, 10: 86, 33: 87, 53: 88, 18: 89, 43: 90, 1: 91, 31: 92, 2: 93, 54: 94, 93: 95, 68: 96, 83: 97, 42: 98, 75: 99}
Numerical columns: ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22',

2022-03-17 00:19:49,978	INFO trial_runner.py:488 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2022-03-17 00:19:49,983	WARNING trial_runner.py:601 -- Attempting to resume experiment from /home/uriel/Documentos/CIC/DCC/Papers/Tabular Transformer/transformer-tabular-data/helena/rnn/checkpoint/param_search. This will ignore any new changes to the specification.
2022-03-17 00:19:49,996	ERROR trial_runner.py:359 -- [Errno 13] Permission denied: '/app'
2022-03-17 00:19:49,997	ERROR trial_runner.py:360 -- Runner restore failed.
Traceback (most recent call last):
  File "/home/uriel/Miniconda3/envs/DCC-attn/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 355, in __init__
    self.resume(run_errored_only=errored_only)
  File "/home/uriel/Miniconda3/envs/DCC-attn/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 611, in resume
    trials = load_trials_from_experiment_checkpoint(runner_state)
  File "/home/uriel/Minicon

In [5]:
results

{'adult': {'cls': {'balanced_accuracy': 0.7877334649080581,
   'roc_auc': 0.7877334649080582},
  'concatenate': {'balanced_accuracy': 0.7616764309459222,
   'roc_auc': 0.7616764309459223},
  'rnn': {'balanced_accuracy': 0.7651785225498691,
   'roc_auc': 0.7651785225498691}},
 'jasmine': {'cls': {'balanced_accuracy': 0.7881701631701632,
   'roc_auc': 0.7881701631701632},
  'rnn': {'balanced_accuracy': 0.8027389277389277,
   'roc_auc': 0.8027389277389276}},
 'ldpa': {'cls': {'balanced_accuracy': 0.5788809939568003},
  'concatenate': {'balanced_accuracy': 0.619408986968123},
  'rnn': {'balanced_accuracy': 0.6332033674046896}}}